In [4]:
import os
import csv
import numpy as np
from keras.layers import Input, Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Cropping2D, Conv2D
from keras.layers.core import Dropout
import keras
from keras.models import Sequential, load_model
from keras.callbacks import TensorBoard
from PIL import Image
import io
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import boto3
import sys, getopt
# import matplotlib
# matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
# import matplotlib.pyplot as plt
import glob
import tensorflow as tf
s3 = boto3.resource('s3')

Using TensorFlow backend.


In [5]:
print(tf.__version__)

1.3.0


In [34]:
def load_data_from_s3(path):
    data =[]

    with open('./csv_classes.csv') as csvfile:
        reader = csv.reader(csvfile)
        line_count = 0
        for line in reader:
            if line_count>0:
                image = line[0]
                color = line[1]
                data.append(([image,color]))
               
            line_count+=1
    return data


def get_image(bucket,sample,resize):
    try:
        key= sample[0]
        image_obj = s3.Object(bucket, key)
        image = io.BytesIO(image_obj.get()['Body'].read())
        image = np.asarray(Image.open(image).resize((resize,resize)))
#         image= np.asarray( Image.open(image), dtype="int32" )
        return image
    except Exception as err:
        print(err)


def generator(bucket,samples,n_classes, batch_size = 32, resize=32):
    while True:
        for offset in range(0,len(samples), batch_size):
            start = offset
            end = offset+batch_size
            batch_sample = samples[start:end]
            images=[]
            colors=[]
            for sample in batch_sample:
                #get images
                image = get_image(bucket, sample, resize)
                images.append(image)
                colors.extend(keras.utils.to_categorical(sample[1], n_classes))

            x = np.array(images)
            y = np.array(colors)
            yield shuffle(x,y)

In [35]:
def run_model(bucket, train_data,validation_data, num_classes,epochs, model_name, resize=32):
    train_generator = generator(bucket, train_data,num_classes , resize=resize)
    val_generator = generator(bucket, validation_data,num_classes, resize=resize)
    
    
    model = Sequential()
    model.add(Lambda(lambda train_generator:train_generator, input_shape=(resize,resize,3)))

    model.add(Conv2D(6, kernel_size=(5,5), activation='relu',init='he_normal'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(16, kernel_size=(5,5), activation='relu',init='he_normal'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    model.add(Dense(120, activation='relu',init='he_normal', name="d1"))
    model.add(Dropout(0.5))

    model.add(Dense(84,activation='relu',init='he_normal', name = "d2"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes,activation='softmax',init='he_normal', name="d3"))
#     optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(optimizer='adamax', metrics=['accuracy', 'top_k_categorical_accuracy'], loss='categorical_crossentropy')
    tensorbrd = TensorBoard('./logs/{}'.format(model_name))
    model.fit_generator(train_generator, steps_per_epoch=int(len(train_data)/32),epochs=epochs,\
                        validation_steps=int(len(validation_data)/32),validation_data = val_generator,\
                        verbose=1, initial_epoch=0, callbacks=[tensorbrd])

    
    
    return model

In [36]:
# Red: 0, Yellow : 1, Green 2

def start():
    bucket = 'capstone-trafficlights'
    classes = ['Red', 'Yellow', 'Green']
    model_name= 'model_v9'
    resize = 32
    epochs=10
    try:
        print("\u2713 Bucket recieved: {}".format(bucket))
        print(" Loading Data ... ")
        # create a directory to store driving_log.csv of the target bucke
        if not os.path.exists(bucket):
            data = load_data_from_s3(bucket)
            train_data, validation_data = train_test_split(data, test_size=0.2)
    
            model = run_model(bucket, train_data, validation_data, len(classes), epochs,model_name=model_name, resize=resize)
            model.save('./{}.h5'.format(model_name))
    except Exception as err:
        print(err)

In [37]:
start()

\u2713 Bucket recieved: capstone-trafficlights
 Loading Data ... 


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, kernel_initializer="he_normal", activation="relu", kernel_size=(5, 5))`
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, kernel_initializer="he_normal", activation="relu", kernel_size=(5, 5))`
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(120, kernel_initializer="he_normal", activation="relu", name="d1")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(84, kernel_initializer="he_normal", activation="relu", name="d2")`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, kerne

Epoch 1/10
136/136 [==============================] - 651s - loss: 2.3470 - acc: 0.8339 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7695 - val_acc: 0.9449 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/10
136/136 [==============================] - 642s - loss: 0.9791 - acc: 0.9246 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.6401 - val_acc: 0.9494 - val_top_k_categorical_accuracy: 1.0000
Epoch 3/10
136/136 [==============================] - 650s - loss: 0.8076 - acc: 0.9318 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.5219 - val_acc: 0.9513 - val_top_k_categorical_accuracy: 1.0000
Epoch 4/10
136/136 [==============================] - 1006s - loss: 0.6191 - acc: 0.9455 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.2562 - val_acc: 0.9724 - val_top_k_categorical_accuracy: 1.0000
Epoch 5/10
136/136 [==============================] - 1073s - loss: 0.5523 - acc: 0.9458 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.3217 - val_acc: 0.9642 - val_top_k_categorical_